1. Install Packages
Installed all required libraries: transformers for emotion detection, sentence-transformers for RAG embeddings, FAISS for similarity search, and Streamlit for web interface.

In [1]:
# ============================
# 1. Install Required Packages
# ============================
!pip install transformers datasets sentence-transformers faiss-cpu streamlit pandas numpy torch
!pip install pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.0 MB/s eta 0:00:00


2. Import Libraries
Imported necessary modules for data processing, ML models, embeddings, and web app development.

In [2]:
# ============================
# 2. Import Libraries
# ============================
import pandas as pd
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import streamlit as st
from pyngrok import ngrok

3. Load Dataset
Loaded tweet_eval emotion dataset from Hugging Face with by example 4 emotion categories: anger, joy, optimism, sadness.

In [3]:
# ============================
# 3. Load Emotion Dataset from tweet_eval
# ============================
print("Loading emotion dataset...")
dataset = load_dataset('cardiffnlp/tweet_eval', 'emotion')
train_data = dataset['train']
test_data = dataset['test']

# Convert to pandas for easier handling
df_train = pd.DataFrame({
    'text': train_data['text'],
    'label': train_data['label']
})

df_test = pd.DataFrame({
    'text': test_data['text'],
    'label': test_data['label']
})

# Map label numbers to emotion names
label_names = {
    0: 'anger',
    1: 'joy',
    2: 'optimism',
    3: 'sadness'
}

df_train['emotion'] = df_train['label'].map(label_names)
df_test['emotion'] = df_test['label'].map(label_names)

print(f"Training samples: {len(df_train)}")
print(f"Test samples: {len(df_test)}")
print("\nEmotion distribution:")
print(df_train['emotion'].value_counts())

Loading emotion dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Training samples: 3257
Test samples: 1421

Emotion distribution:
emotion
anger       1400
sadness      855
joy          708
optimism     294
Name: count, dtype: int64


4. Emotion Detection Model
Used pre-trained DistilBERT model fine-tuned on emotion classification to detect emotions from user text inputs.

In [4]:
# ============================
# 4. Load Pre-trained Emotion Detection Model
# ============================
print("\nLoading emotion detection model...")
emotion_classifier = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    return_all_scores=False
)

# Test the emotion detector
test_texts = [
    "I'm so happy today!",
    "This is terrible and I'm very sad",
    "I'm really angry about this situation",
    "I feel optimistic about the future"
]

print("\nTesting emotion detection:")
for text in test_texts:
    result = emotion_classifier(text)[0]
    print(f"Text: {text}")
    print(f"Detected emotion: {result['label']} (confidence: {result['score']:.3f})")
    print()


Loading emotion detection model...


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(



Testing emotion detection:
Text: I'm so happy today!
Detected emotion: joy (confidence: 0.999)

Text: This is terrible and I'm very sad
Detected emotion: sadness (confidence: 0.998)

Text: I'm really angry about this situation
Detected emotion: anger (confidence: 0.998)

Text: I feel optimistic about the future
Detected emotion: joy (confidence: 0.999)



5. Create RAG System
Built response database with empathetic templates for each emotion. Used sentence transformers to create embeddings and FAISS for fast similarity search.

In [5]:
# ============================
# 5. Create RAG Corpus from Emotion Data
# ============================
print("Creating RAG corpus...")

# Create empathetic responses for each emotion
empathetic_responses = {
    'anger': [
        "I understand you're feeling angry. It's okay to feel this way. Would you like to talk about what's bothering you?",
        "Anger can be really overwhelming. Remember to take deep breaths. I'm here to listen.",
        "It sounds like you're really frustrated. Sometimes expressing these feelings can help."
    ],
    'joy': [
        "That's wonderful! I'm so happy to hear you're feeling joyful!",
        "Your happiness is contagious! What's bringing you so much joy today?",
        "It's great to hear you're experiencing joy! Celebrate these positive moments!"
    ],
    'optimism': [
        "Your optimism is inspiring! Keep looking at the bright side of things.",
        "It's great that you're feeling optimistic! Positive thinking can make a big difference.",
        "I love your optimistic outlook! What's making you feel so hopeful?"
    ],
    'sadness': [
        "I'm really sorry you're feeling sad. I'm here for you if you want to talk.",
        "It's okay to feel sad sometimes. These feelings will pass. Would you like to share what's on your mind?",
        "I hear your sadness. Remember that you're not alone, and I'm here to support you."
    ],
    'neutral': [
        "Thanks for sharing. I'm here to listen if you want to talk more.",
        "I appreciate you opening up. How are you really feeling about this?",
        "Thank you for telling me. Would you like to explore this further?"
    ]
}

# Create RAG corpus
rag_corpus = []
for emotion, responses in empathetic_responses.items():
    for response in responses:
        rag_corpus.append({
            'emotion': emotion,
            'response': response
        })

rag_df = pd.DataFrame(rag_corpus)
print(f"Created RAG corpus with {len(rag_df)} empathetic responses")

Creating RAG corpus...
Created RAG corpus with 15 empathetic responses


6. RAG System Setup
Creates embeddings for all responses using sentence transformers. Uses FAISS for fast similarity search to find the most relevant empathetic response based on user's emotion and input text.

Key Steps:

Convert responses to numerical embeddings

Build FAISS index for efficient searching

Retrieve best-matching response that matches detected emotion

Fallback to most similar response if no emotion match found

In [6]:
# ============================
# 6. Set up RAG with Sentence Transformers and FAISS
# ============================
print("Setting up RAG system...")

# Initialize sentence transformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for all responses
response_texts = rag_df['response'].tolist()
response_embeddings = embedder.encode(response_texts, convert_to_numpy=True)

# Normalize embeddings for cosine similarity
response_embeddings = response_embeddings / np.linalg.norm(response_embeddings, axis=1, keepdims=True)

# Create FAISS index
dimension = response_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(response_embeddings.astype('float32'))

def retrieve_empathetic_response(user_input, detected_emotion, top_k=3):
    """Retrieve the most appropriate empathetic response"""
    # Get embedding for user input
    input_embedding = embedder.encode([user_input], convert_to_numpy=True)
    input_embedding = input_embedding / np.linalg.norm(input_embedding)

    # Search in FAISS index
    D, I = index.search(input_embedding.astype('float32'), top_k)

    # Filter responses by detected emotion and get the best match
    best_response = None
    best_score = -1

    for i, idx in enumerate(I[0]):
        response_emotion = rag_df.iloc[idx]['emotion']
        response_text = rag_df.iloc[idx]['response']
        similarity_score = D[0][i]

        # Prioritize responses that match the detected emotion
        if response_emotion == detected_emotion and similarity_score > best_score:
            best_response = response_text
            best_score = similarity_score

    # Fallback: if no emotion match, return the most similar response
    if best_response is None and len(I[0]) > 0:
        best_response = rag_df.iloc[I[0][0]]['response']

    return best_response


Setting up RAG system...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

7. Few-Shot Prompt Engineering
Combines emotion detection with RAG retrieval to generate context-aware responses. Uses the detected emotion to select the most appropriate empathetic response from the database and adds a safety disclaimer.

Workflow:

Detects emotion from user input using classifier

Retrieves matching empathetic response using RAG system

Appends professional help disclaimer

Returns structured response with emotion and confidence data

In [7]:
# ============================
# 7. Few-Shot Prompt Engineering
# ============================
def create_empathetic_response(user_input):
    """Main function to create empathetic response using few-shot prompting"""

    # Detect emotion
    emotion_result = emotion_classifier(user_input)[0]
    detected_emotion = emotion_result['label'].lower()
    confidence = emotion_result['score']

    # Retrieve appropriate response using RAG
    empathetic_response = retrieve_empathetic_response(user_input, detected_emotion)

    # Add disclaimer
    disclaimer = " Note: I'm not a therapist; please seek professional help for serious issues."

    # Construct final response with few-shot context
    final_response = f"{empathetic_response}{disclaimer}"

    return {
        'user_input': user_input,
        'detected_emotion': detected_emotion,
        'confidence': confidence,
        'response': final_response
    }

8. System Testing
Tests the complete pipeline with 10 diverse emotional inputs. Compares detected emotions against expected labels to calculate accuracy.

Testing Process:

Runs 10 sample messages through emotion detection + RAG response

Compares predicted vs expected emotions for accuracy measurement

Displays detailed results for each test case

Calculates and reports final system accuracy (target: 80%+)

In [8]:
# ============================
# 8. Test the System
# ============================
print("\nTesting the complete system...")
test_messages = [
    "I'm feeling really happy today!",
    "This is the worst day of my life",
    "I'm so angry I could scream",
    "I feel hopeful about the future",
    "Everything seems pointless",
    "I'm excited about my new job",
    "I'm frustrated with this situation",
    "I feel completely lost",
    "This is amazing news!",
    "I'm worried about what might happen"
]

print("Running tests on 10 messages:")
results = []
for i, message in enumerate(test_messages, 1):
    result = create_empathetic_response(message)
    results.append(result)
    print(f"\nTest {i}:")
    print(f"Input: {result['user_input']}")
    print(f"Detected emotion: {result['detected_emotion']} ({result['confidence']:.3f})")
    print(f"Response: {result['response']}")

# Calculate accuracy
correct_predictions = 0
expected_emotions = ['joy', 'sadness', 'anger', 'optimism', 'sadness',
                    'joy', 'anger', 'sadness', 'joy', 'sadness']

for i, (result, expected) in enumerate(zip(results, expected_emotions)):
    if result['detected_emotion'] == expected:
        correct_predictions += 1
    else:
        print(f"Mismatch in test {i+1}: Expected {expected}, got {result['detected_emotion']}")

accuracy = correct_predictions / len(test_messages)
print(f"\nAccuracy: {accuracy:.1%} ({correct_predictions}/{len(test_messages)})")



Testing the complete system...
Running tests on 10 messages:

Test 1:
Input: I'm feeling really happy today!
Detected emotion: joy (0.999)
Response: That's wonderful! I'm so happy to hear you're feeling joyful! Note: I'm not a therapist; please seek professional help for serious issues.

Test 2:
Input: This is the worst day of my life
Detected emotion: sadness (0.974)
Response: I hear your sadness. Remember that you're not alone, and I'm here to support you. Note: I'm not a therapist; please seek professional help for serious issues.

Test 3:
Input: I'm so angry I could scream
Detected emotion: anger (0.998)
Response: Anger can be really overwhelming. Remember to take deep breaths. I'm here to listen. Note: I'm not a therapist; please seek professional help for serious issues.

Test 4:
Input: I feel hopeful about the future
Detected emotion: joy (0.999)
Response: I love your optimistic outlook! What's making you feel so hopeful? Note: I'm not a therapist; please seek professional help

**10. Streamlit Web App Deployment**  
Creates and launches interactive web interface on port 7860. Builds real-time chat application with emotion detection, response generation, and conversation history.

**Features:**  
- Real-time emotion detection and empathetic responses  
- Chat history preservation and display  
- Emotion confidence scoring  
- Professional disclaimer integration  
- Responsive web interface with sidebar information

In [15]:
# ============================
# 10. Run Streamlit on Port 7860
# ============================
print("Setting up Streamlit on port 7860...")

# First, write the complete corrected app.py
app_code = '''
import streamlit as st
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss

# Initialize models (will be loaded once)
@st.cache_resource
def load_models():
    # Emotion classifier
    emotion_classifier = pipeline(
        "text-classification",
        model="bhadresh-savani/distilbert-base-uncased-emotion",
        return_all_scores=False
    )

    # Sentence transformer for RAG
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    return emotion_classifier, embedder

@st.cache_resource
def setup_rag(_embedder):
    # Empathetic responses corpus
    empathetic_responses = {
        'anger': [
            "I understand you're feeling angry. It's okay to feel this way. Would you like to talk about what's bothering you?",
            "Anger can be really overwhelming. Remember to take deep breaths. I'm here to listen.",
            "It sounds like you're really frustrated. Sometimes expressing these feelings can help."
        ],
        'joy': [
            "That's wonderful! I'm so happy to hear you're feeling joyful!",
            "Your happiness is contagious! What's bringing you so much joy today?",
            "It's great to hear you're experiencing joy! Celebrate these positive moments!"
        ],
        'optimism': [
            "Your optimism is inspiring! Keep looking at the bright side of things.",
            "It's great that you're feeling optimistic! Positive thinking can make a big difference.",
            "I love your optimistic outlook! What's making you feel so hopeful?"
        ],
        'sadness': [
            "I'm really sorry you're feeling sad. I'm here for you if you want to talk.",
            "It's okay to feel sad sometimes. These feelings will pass. Would you like to share what's on your mind?",
            "I hear your sadness. Remember that you're not alone, and I'm here to support you."
        ],
        'neutral': [
            "Thanks for sharing. I'm here to listen if you want to talk more.",
            "I appreciate you opening up. How are you really feeling about this?",
            "Thank you for telling me. Would you like to explore this further?"
        ]
    }

    # Create RAG corpus
    rag_corpus = []
    for emotion, responses in empathetic_responses.items():
        for response in responses:
            rag_corpus.append({
                'emotion': emotion,
                'response': response
            })

    rag_df = pd.DataFrame(rag_corpus)

    # Create embeddings
    response_texts = rag_df['response'].tolist()
    response_embeddings = _embedder.encode(response_texts, convert_to_numpy=True)
    response_embeddings = response_embeddings / np.linalg.norm(response_embeddings, axis=1, keepdims=True)

    # Create FAISS index
    dimension = response_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(response_embeddings.astype('float32'))

    return rag_df, index

def retrieve_empathetic_response(_embedder, index, rag_df, user_input, detected_emotion, top_k=3):
    """Retrieve empathetic response using RAG"""
    input_embedding = _embedder.encode([user_input], convert_to_numpy=True)
    input_embedding = input_embedding / np.linalg.norm(input_embedding)

    D, I = index.search(input_embedding.astype('float32'), top_k)

    best_response = None
    best_score = -1

    for i, idx in enumerate(I[0]):
        response_emotion = rag_df.iloc[idx]['emotion']
        response_text = rag_df.iloc[idx]['response']
        similarity_score = D[0][i]

        if response_emotion == detected_emotion and similarity_score > best_score:
            best_response = response_text
            best_score = similarity_score

    if best_response is None and len(I[0]) > 0:
        best_response = rag_df.iloc[I[0][0]]['response']

    return best_response

def main():
    st.set_page_config(
        page_title="EmpathyBot - Sentiment-Driven Chat",
        page_icon="🤗",
        layout="wide"
    )

    st.title("🤗 EmpathyBot - Sentiment-Driven Chat")
    st.markdown("""
    I'm here to listen and respond with empathy. Share how you're feeling, and I'll do my best to understand and support you.
    """)

    # Load models
    emotion_classifier, embedder = load_models()
    rag_df, faiss_index = setup_rag(embedder)

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            if "emotion" in message:
                st.caption(f"Detected emotion: {message['emotion']}")

    # Chat input
    if prompt := st.chat_input("How are you feeling today?"):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})

        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)

        # Generate response
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                # Detect emotion
                emotion_result = emotion_classifier(prompt)[0]
                detected_emotion = emotion_result['label'].lower()
                confidence = emotion_result['score']

                # Retrieve empathetic response
                empathetic_response = retrieve_empathetic_response(
                    embedder, faiss_index, rag_df, prompt, detected_emotion
                )

                # Add disclaimer
                disclaimer = " Note: I'm not a therapist; please seek professional help for serious issues."
                full_response = f"{empathetic_response}{disclaimer}"

                # Display response
                st.markdown(full_response)
                st.caption(f"Detected emotion: {detected_emotion} ({confidence:.2f} confidence)")

        # Add assistant response to chat history
        st.session_state.messages.append({
            "role": "assistant",
            "content": full_response,
            "emotion": detected_emotion
        })

    # Sidebar with information
    with st.sidebar:
        st.header("About EmpathyBot")
        st.markdown("""
        - **Emotion Detection**: Powered by DistilBERT fine-tuned on emotion classification
        - **RAG System**: Uses sentence transformers and FAISS for empathetic response retrieval
        - **Few-Shot Prompting**: Combines emotion detection with contextual responses

        **Disclaimer**: This is a prototype. For serious issues, please seek professional help.
        """)

        if st.button("Clear Chat History"):
            st.session_state.messages = []
            st.rerun()

if __name__ == "__main__":
    main()
'''

# Write the app.py file
with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ app.py created successfully!")

# Now run Streamlit on port 7860
print("Starting Streamlit on port 7860...")

# Kill any process on port 7860
!pkill -f "streamlit.*7860" || true
!fuser -k 7860/tcp 2>/dev/null || true

# Run Streamlit in the background
import subprocess
import time
import threading

def run_streamlit():
    process = subprocess.Popen([
        'streamlit', 'run', 'app.py',
        '--server.port', '7860',
        '--server.headless', 'true',
        '--server.enableCORS', 'true',
        '--browser.gatherUsageStats', 'false'
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Read output for a while to check for errors
    time.sleep(3)
    stdout, stderr = process.communicate(timeout=5)

    if stderr:
        print(f"Streamlit errors: {stderr.decode()}")
    if stdout:
        print(f"Streamlit output: {stdout.decode()}")

# Run in a separate thread
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# Wait for Streamlit to start
time.sleep(10)

print("✅ Streamlit app should be running on port 7860")
print("🌐 Local URL: http://localhost:7860")
print("📋 Check if it's working by visiting the URL above")

# Test if the port is open
import socket
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', 7860))
    if result == 0:
        print("✅ Port 7860 is open and accepting connections")
    else:
        print("❌ Port 7860 is not accessible - trying alternative approach...")
        # Try running directly
        !streamlit run app.py --server.port 7860 --server.headless true &
        time.sleep(8)
    sock.close()
except Exception as e:
    print(f"❌ Socket test failed: {e}")

# Show running processes to verify
print("\n📊 Checking running processes:")
!ps aux | grep streamlit | grep -v grep || echo "No streamlit processes found"

# Alternative: Check if we can access the port
print("\n🌐 Trying to access the app...")
try:
    import requests
    response = requests.get('http://localhost:7860', timeout=5)
    print(f"✅ App is responding with status code: {response.status_code}")
except Exception as e:
    print(f"❌ Cannot access app: {e}")
    print("📋 Try running this command manually:")
    print("!streamlit run app.py --server.port 7860 --server.headless true")

Setting up Streamlit on port 7860...
✅ app.py created successfully!
Starting Streamlit on port 7860...
^C


Exception in thread Thread-6 (run_streamlit):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2493021276.py", line 220, in run_streamlit
  File "/usr/lib/python3.12/subprocess.py", line 1209, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 2116, in _communicate
    self._check_timeout(endtime, orig_timeout, stdout, stderr)
  File "/usr/lib/python3.12/subprocess.py", line 1253, in _check_timeout
    raise TimeoutExpired(
subprocess.TimeoutExpired: Command '['streamlit', 'run', 'app.py', '--server.port', '7860', '--server.headless', 'true', '--server.enableCORS', 'true', '--browser.gatherUsageStats', 'false']' timed out after 5 seconds


✅ Streamlit app should be running on port 7860
🌐 Local URL: http://localhost:7860
📋 Check if it's working by visiting the URL above
✅ Port 7860 is open and accepting connections

📊 Checking running processes:
root        1887 20.0  0.5  87468 71288 ?        S    16:27   0:02 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 7860 --server.headless true --server.enableCORS true --browser.gatherUsageStats false

🌐 Trying to access the app...
✅ App is responding with status code: 200


In [16]:
from pyngrok import ngrok, conf

# Replace with your token
NGROK_AUTH_TOKEN = "32QCuqces3fFsY2swBD36T0CNBk_5riNg8ZVXmpNt9HGxJ6Wi"

!ngrok config add-authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
!pip install streamlit pyngrok

from pyngrok import ngrok
import os

# Kill any previous tunnels
ngrok.kill()

# Start a tunnel on port 8501
public_url = ngrok.connect(7860)
print("Streamlit URL:", public_url)

# Run Streamlit app
!streamlit run app.py --server.port 7860 &> /dev/null&


Streamlit URL: NgrokTunnel: "https://7d9f91aacaee.ngrok-free.app" -> "http://localhost:7860"
